In [5]:
from helpers import NormalizedEnv, RandomAgent

In [6]:
import random
import torch
import torch.nn as nn
import torch.nn.functional as F

class ReplayBuffer:
    def __init__(self, max_size):
        self.max_size = max_size
        self.buffer = []
        self.idx = 0

    def __len__(self):
        return len(self.buffer)

    def add(self, state, action, reward, next_state, trunc):
        transition = (state, action, reward, next_state, trunc)
        if len(self.buffer) < self.max_size:
            self.buffer.append(transition)
        else:
            # not sure about the behaviour when buffer is overloaded.
            self.buffer[self.idx] = transition
            self.idx = (self.idx + 1) % self.max_size

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, truncs = zip(*batch)
        return states, actions, rewards, next_states, truncs
    
class QNetwork(nn.Module):
    def __init__(self):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(4, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 1)

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def update(self, optimizer, transitions, gamma):
        states, actions, rewards, next_states, truncs = transitions
        states = torch.FloatTensor(states)
        actions = torch.FloatTensor(actions)
        rewards = torch.FloatTensor(rewards)
        next_states = torch.FloatTensor(next_states)
        truncs = torch.FloatTensor(truncs)

        # Compute the TD target
        with torch.no_grad():
            next_actions = policy.select_action(next_states)
            q_next = self.forward(torch.cat([next_states, next_actions], dim=1))
            target = rewards + gamma * q_next * (1 - truncs)

        # Compute the TD error and update the Q network
        q_values = self.forward(torch.cat([states, actions], dim=1))
        loss = F.mse_loss(q_values, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        return loss.item()

In [7]:
# Implementation of a heuristic policy for the pendulum

class HeuristicPendulumAgent:
    def __init__(self, env):
        self.state_size = env.observation_space.shape[0]
        self.action_size = env.action_space.shape[0]
        self.env = env
        
    def compute_action(self, state):
        _, y, v = state
        torque = self.env.m*self.env.g*self.env.l*y # fixed torque
        action = np.empty((1,))
        
        if (y < 0):
            np.append(action, np.sign(v)*torque) # same direction to angular velocity
        else:
            np.append(action, (-1)*np.sign(v)*torque) # opposite direction to angular velocity
        return action

In [8]:
env = gym.make("Pendulum-v1")
norm_env = NormalizedEnv(env)
heur_ag = HeuristicPendulumAgent(norm_env)
buffer_size = 1e4
replay_buffer = ReplayBuffer(buffer_size)
critic = QNetwork()
n_ep = 1000

gamma = 0.99
lr = 1e-4

losses = []

for x in range(n_ep) : 
    states, actions, rewards, next_states, truncs = replay_buffer.sample(128)
    loss = critic.update()
    
    

IndentationError: expected an indented block (2927289266.py, line 4)